# Model Versioning And Information
This notebook is intended to track the model versions. This includes metrics on model performance, hyperparameters, algorithms used, and any additional information related to the training model. Due to limited resources in AWS, the models are not trained within AWS and are instead trained on local hardward. The information for the Model groups are generated from uploaded h5 files. 

In [5]:
import boto3
import os
import sagemaker
import pandas as pd 
import numpy as np 
import time

In [2]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
 
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

# Create s3 bucket for models
s3_model_path = 's3://{}/group-5/models'.format(bucket)
print(s3_model_path)

s3://sagemaker-us-east-1-399018723364/group-5/models


In [3]:
!aws s3 cp 'fer_best_model.h5' $s3_model_path/

upload: ./fer_best_model.h5 to s3://sagemaker-us-east-1-399018723364/group-5/models/fer_best_model.h5


In [6]:
sagemaker_client = boto3.client('sagemaker')
model_pacakge_group_name = "basic-cnn" + str(round(time.time()))
model_pacakge_group_description = ("Simple CNN model group for FER-2013 dataset")

# Create model package group
response = sagemaker_client.create_model_package_group(
    ModelPackageGroupName=model_pacakge_group_name,
    ModelPackageGroupDescription=model_pacakge_group_description
)

print(f"Creation of Model Package Group: {response}")

Creation of Model Package Group: {'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:399018723364:model-package-group/basic-cnn1738537479', 'ResponseMetadata': {'RequestId': '9ac1961e-65d7-4325-89db-aff845cbccd7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9ac1961e-65d7-4325-89db-aff845cbccd7', 'content-type': 'application/x-amz-json-1.1', 'content-length': '107', 'date': 'Sun, 02 Feb 2025 23:04:38 GMT'}, 'RetryAttempts': 0}}


In [10]:
!aws ecr list-images --region us-east-1


usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help

aws: error: the following arguments are required: --repository-name



In [15]:
# Define model and version info
s3_model_path = f's3://sagemaker-us-east-1-399018723364/group-5/models/fer_best_model.h5'
# Create model package for versioning
model_package_name = 'v1-basic-fer-cnn'
## FIXME Find the right image to use, don't want xgboost, want tensorflow but everything was giving access denied
container_image_uri = "811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:1"
model_package_response = sagemaker_client.create_model_package(
    ModelPackageName=model_package_name,
    ModelPackageDescription="Version 1 of CNN model",
    InferenceSpecification={
    "Containers": [
        {
            "Image": container_image_uri,
            "ModelDataUrl": s3_model_path,
        }
    ],
    "SupportedContentTypes": ["text/csv"],
    "SupportedResponseMIMETypes": ["text/csv"],
    "SupportedTransformInstanceTypes": ['ml.m4.xlarge'],
    "SupportedRealtimeInferenceInstanceTypes": ['ml.t2.xlarge']
    }
)
print(model_package_response)

{'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:399018723364:model-package/v1-basic-fer-cnn', 'ResponseMetadata': {'RequestId': '03feab7f-3917-440a-96a7-93d8f7b551bd', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '03feab7f-3917-440a-96a7-93d8f7b551bd', 'content-type': 'application/x-amz-json-1.1', 'content-length': '93', 'date': 'Sun, 02 Feb 2025 23:25:00 GMT'}, 'RetryAttempts': 0}}


In [16]:
# Describe Model Package
describe_model_pacakge_response = sagemaker_client.describe_model_package(
    ModelPackageName=model_package_name
)

print(f"Model Package Details: {describe_model_pacakge_response}")

Model Package Details: {'ModelPackageName': 'v1-basic-fer-cnn', 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:399018723364:model-package/v1-basic-fer-cnn', 'ModelPackageDescription': 'Version 1 of CNN model', 'CreationTime': datetime.datetime(2025, 2, 2, 23, 25, 0, 251000, tzinfo=tzlocal()), 'InferenceSpecification': {'Containers': [{'Image': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:1', 'ImageDigest': 'sha256:0c8f830ac408e6dee08445fb60392e9c3f05f790a4b3c07ec22327c08f75bcbf', 'ModelDataUrl': 's3://sagemaker-us-east-1-399018723364/group-5/models/fer_best_model.h5'}], 'SupportedTransformInstanceTypes': ['ml.m4.xlarge'], 'SupportedRealtimeInferenceInstanceTypes': ['ml.t2.xlarge'], 'SupportedContentTypes': ['text/csv'], 'SupportedResponseMIMETypes': ['text/csv']}, 'ModelPackageStatus': 'Completed', 'ModelPackageStatusDetails': {'ValidationStatuses': [], 'ImageScanStatuses': []}, 'CertifyForMarketplace': False, 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:39901

In [18]:
import json
model_content = {
    "model_overview": {
        "model_description": "Basic CNN model",
        "model_creator": "Group 5",
        "model_artifact": [s3_model_path],
        "algorithm_type": "CNN",
    },
    "training_details": {
        "objective_function": "identify malignent masses given input data",
        "training_observations": "35000",
    },
}
response = sagemaker_client.create_model_card(
    ModelCardName="basic-cnn-model-card",
    Content=json.dumps(model_content),
    ModelCardStatus='Draft',
)

print(f"Model card created: {response}")

Model card created: {'ModelCardArn': 'arn:aws:sagemaker:us-east-1:399018723364:model-card/basic-cnn-model-card', 'ResponseMetadata': {'RequestId': 'a20db295-b1b5-44c3-96c6-bc955fdbf998', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a20db295-b1b5-44c3-96c6-bc955fdbf998', 'content-type': 'application/x-amz-json-1.1', 'content-length': '91', 'date': 'Sun, 02 Feb 2025 23:28:26 GMT'}, 'RetryAttempts': 0}}


In [19]:
# Describe Model Card (for submission screenshot)
describe_model_card_response = sagemaker_client.describe_model_card(
    ModelCardName="basic-cnn-model-card"
)
print("Model Card Details:", describe_model_card_response)

Model Card Details: {'ModelCardArn': 'arn:aws:sagemaker:us-east-1:399018723364:model-card/basic-cnn-model-card', 'ModelCardName': 'basic-cnn-model-card', 'ModelCardVersion': 1, 'Content': '{"model_overview": {"model_description": "Basic CNN model", "model_creator": "Group 5", "model_artifact": ["s3://sagemaker-us-east-1-399018723364/group-5/models/fer_best_model.h5"], "algorithm_type": "CNN"}, "training_details": {"objective_function": "identify malignent masses given input data", "training_observations": "35000"}}', 'ModelCardStatus': 'Draft', 'CreationTime': datetime.datetime(2025, 2, 2, 23, 28, 26, 621000, tzinfo=tzlocal()), 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:399018723364:user-profile/d-coj0j6t5xpww/kvierczhalek', 'UserProfileName': 'kvierczhalek', 'DomainId': 'd-coj0j6t5xpww'}, 'LastModifiedTime': datetime.datetime(2025, 2, 2, 23, 28, 26, 621000, tzinfo=tzlocal()), 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:399018723364:user-profile/